In [ ]:
# %pip install --upgrade --user --quiet google-cloud-aiplatform

In [ ]:
# # Restart the kernel after installing packages:

# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [1]:
PROJECT_ID = "esg-satelite"
REGION = "europe-west2"

# !gcloud auth application-default login --quiet
!gcloud auth application-default set-quota-project {PROJECT_ID}
!Y | gcloud config set project {PROJECT_ID}


Credentials saved to file: [/home/jupyter/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "esg-satelite" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.
/bin/bash: line 1: Y: command not found
Updated property [core/project].


In [11]:
%%bash

gcloud config set project esg-satelite

Updated property [core/project].


In [2]:
from vertexai.generative_models import (
    GenerativeModel,
    GenerationResponse,
    GenerationConfig,
    Tool,
    grounding,
)
from vertexai.preview.generative_models import grounding as preview_grounding
from IPython.display import display, Markdown

def print_grounding_response(response: GenerationResponse):
    """Prints Gemini response with grounding citations."""
    grounding_metadata = response.candidates[0].grounding_metadata

    # Citation indices are in byte units
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)

    prev_index = 0
    markdown_text = ""

    sources: dict[str, str] = {}
    footnote = 1
    for attribution in grounding_metadata.grounding_attributions:
        context = attribution.web or attribution.retrieved_context
        if not context:
            print(f"Skipping Grounding Attribution {attribution}")
            continue

        title = context.title
        uri = context.uri
        end_index = int(attribution.segment.end_index)

        if uri not in sources:
            sources[uri] = {"title": title, "footnote": footnote}
            footnote += 1

        text_segment = text_bytes[prev_index:end_index].decode(ENCODING)
        markdown_text += f"{text_segment} [[{sources[uri]['footnote']}]]({uri})"
        prev_index = end_index

    if prev_index < len(text_bytes):
        markdown_text += str(text_bytes[prev_index:], encoding=ENCODING)

    markdown_text += "\n## Grounding Sources\n"

    if grounding_metadata.web_search_queries:
        markdown_text += (
            f"\n**Web Search Queries:** {grounding_metadata.web_search_queries}\n"
        )
    elif grounding_metadata.retrieval_queries:
        markdown_text += (
            f"\n**Retrieval Queries:** {grounding_metadata.retrieval_queries}\n"
        )

    for uri, source in sources.items():
        markdown_text += f"{source['footnote']}. [{source['title']}]({uri})\n"

    display(Markdown(markdown_text))


import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

In [3]:
model = GenerativeModel(
    model_name="gemini-1.0-pro-002",
    system_instruction = [
        "You are an industry researcher who writes industry/market trend research report given latest news, websites and reports/documents. Specifically, the end users are interested in knowing the opportunies.", 
        # "Provide statements/citations that reflect a deep understanding of the field, guaranteeing their correctness.",
        "Any statements/citations made in your report would require a reference together of its URI of where the conlcusion was driven from.",
        # "For example, the market increase by 1% in 2022 [1], where [1] is the reference for this statement.",
        "Please always give the website URI for each reference/citation."
        "The report produced should be long, summarise all information in a structured way and follow a standard industry research report layout.", 
        "Create markdown tables if necessary."
        ]
    )

In [4]:
prompt = """
User input: Give me the report on the Aisa EV industry. 
Answer:
"""

# 1. No Gounding

In [ ]:
response = model.generate_content(prompt)

display(Markdown(response.text))

# 2. Gounding: Google Search

In [7]:
# Use Google Search for grounding
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

response = model.generate_content(
    prompt,
    tools=[tool],
    generation_config=GenerationConfig(
        temperature=0.0,
        max_output_tokens=8192,
        top_k=40,
        top_p=0.95
    )
)

# display(Markdown(response.text))
# print("-----------------------------------------------------------------")
# print(response.candidates[0].to_dict()["grounding_metadata"])


print_grounding_response(response)

## Asia Electric Vehicle (EV) Industry Report

**Date:** June 26, 2024

**Executive Summary:**

The Asia EV market is experiencing significant growth, driven by government policies, rising fuel prices, and increasing environmental awareness. The market is expected to reach USD 4.70 billion in ASEAN and USD 1186.89 billion in the broader Asia Pacific region by 2029, with a CAGR of 32.73% and 18.06% respectively. 

**Market Size and Growth:**

* The ASEAN EV market size was estimated at USD 0.86 billion in 2023 and is projected to reach USD 4.70 billion by 2029.
* The Asia Pacific EV market size was valued at USD 225.56 billion in 2022 and is expected to reach USD 1186.89 billion by 2032.

**Key Drivers:**

* **Government policies:** Governments in the region are implementing supportive policies, such as subsidies, tax breaks, and charging infrastructure development, to promote EV adoption.
* **Rising fuel prices:** The increasing cost of fossil fuels is making EVs a more attractive option for consumers.
* **Environmental awareness:** Growing concerns about air pollution and climate change are driving demand for EVs.

**Opportunities:**

* **Growing demand for EVs:** The increasing demand for EVs presents significant opportunities for manufacturers, charging infrastructure providers, and other stakeholders in the EV ecosystem.
* **Technological advancements:** Advancements in battery technology, charging infrastructure, and autonomous driving are expected to further accelerate EV adoption.
* **Government support:** Continued government support for the EV industry will create a favorable environment for growth.

**Challenges:**

* **High initial cost:** EVs are typically more expensive than traditional gasoline-powered vehicles.
* **Limited charging infrastructure:** The availability of charging infrastructure is still limited in many parts of Asia.
* **Range anxiety:** Consumers may be concerned about the range of EVs and the availability of charging stations.

**Key Players:**

* **BYD (China)**
* **Tesla (US)**
* **Volkswagen Group (Germany)**
* **Hyundai Motor Group (South Korea)**
* **Nissan (Japan)**

**Conclusion:**

The Asia EV market is poised for significant growth in the coming years. The key drivers of this growth include government policies, rising fuel prices, and increasing environmental awareness. However, challenges such as high initial cost, limited charging infrastructure, and range anxiety need to be addressed to further accelerate EV adoption.

**References:**

* [ASEAN EV Market - Growth, Share, Size & Electric Vehicle Industry - Mordor Intelligence](https://www.mordorintelligence.com/industry-reports/asean-electric-vehicle-market)
* [Asia Pacific Electric vehicle (EV) Market Size, Share, Forecast Report - 2032](https://www.sphericalinsights.com/reports/asia-pacific-electric-vehicle-ev-market)

## Grounding Sources

**Web Search Queries:** ['Aisa EV industry report']


In [10]:
import ipywidgets as widgets
from IPython.display import clear_output

button = widgets.Button(description="Ask Me!")
output = widgets.Output()
button_stp = widgets.Button(description="More details")
output = widgets.Output()
text = widgets.Text(
    description="Question:", layout=widgets.Layout(width="80%", height="50px")
)
display(text, button, button_stp, output)


@output.capture()
def on_button_clicked(b):
    # clear_output()
    # question = text.value

    # result = retrieval_chain.invoke({"input": question})
    # source_documents = list({doc.metadata["source"] for doc in result["context"]})

    # print("\nAnswer-", result["answer"])
    # print("\nSource-", "\n".join(source_documents))
    # print("\n")
    
    print("\nAnswer-", "hello")
    print("\nSource-", "\n".join(["My notes"]))
    print("\n")


@output.capture()
def on_stp_clicked(b):
    # clear_output()
    # question = text.value
    # query = question + "Give detailed information as much as possible. "
    # result = retrieval_chain.invoke({"input": query})
    # source_documents = list({doc.metadata["source"] for doc in result["context"]})

    # print("\nAnswer-", result["answer"])
    # print("\nSource-", "\n".join(source_documents))
    # print("\n")
    
    print("\n1Answer-", "hello")
    print("\n1Source-", "\n".join(["My notes"]))
    print("\n")



button.on_click(on_button_clicked)
button_stp.on_click(on_stp_clicked)

Text(value='', description='Question:', layout=Layout(height='50px', width='80%'))

Button(description='Ask Me!', style=ButtonStyle())

Button(description='More details', style=ButtonStyle())

Output()

In [ ]:
import vertexai

from vertexai.generative_models import GenerativeModel, Part

# TODO(developer): Update and un-comment below lines
# project_id = "PROJECT_ID"

vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel(model_name="gemini-1.5-flash-001")

prompt = """
You are a very professional document summarization specialist.
Please summarize the given document.
"""

pdf_file_uri = "gs://cloud-samples-data/generative-ai/pdf/2403.05530.pdf"
pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
contents = [pdf_file, prompt]

response = model.generate_content(contents)
print(response.text)